# Week 1 - Second Year Project

---

**Learning goals**

- be familiar with the concept of regular expressions
- be able to discuss issues that arise in tokenization and segmentation
- be able to use the Unix command-line tools for navigation, search (`grep`), count (`wc`), and basic text processing (`sed` for substitution), as well as the pipe (`|`), e.g., to count word types or extract a simple word frequency list

**Notebook overview**

*Lecture 1*
1. Regular expressions - get basic familiarity with the concept
2. Tokenization - learn about how to approach tokenization and its challenges, then apply the knowledge to a small example text
3. Twitter tokenization - learn how to tokenize domain specific text
4. Sentence segmentation - learn how to segment given text into sentences

*Lecture 2*

5. Linux Command Line for NLP - learn how to use command line to quickly extract useful information from provided text file
6. Advanced Use of Linux Command Line - construct more complex command to extract word frequency from a text


## 1. Regular Expressions (pen and paper)
For this section, it might be handy to use the website https://regex101.com/ to test your solutions.
Note: By word, we mean any alphabetic string separated from other words by whitespace, any relevant punctuation, line breaks, etc., as defined in [J&M](https://web.stanford.edu/~jurafsky/slp3/old_dec21/). If we do not specify word, any substring match might be sufficient.
- a) Write a regular expression (regex or pattern) that matches any of the following words: `cat`, `sat`, `mat`.
<br>
(Bonus: What is a possible long solution? Can you find a shorter solution? hint: match characters instead of words)
- b) Write a regular expression that matches numbers, e.g. 12, 1,000, 39.95
- c) Expand the previous solution to match Danish prices indications, e.g., `1,000 kr` or `39.95 DKK` or `19.95`.

## 2. Tokenization

(Adapted notebook from S. Riedel, UCL & Facebook: https://github.com/uclnlp/stat-nlp-book).

In Python, a simple way to tokenize a text is via the `split` method that divides a text wherever a particular substring is found. In the code below this pattern is simply the whitespace character, and this seems like a reasonable starting point for an English tokenization approach.

This is analogous to the `sed` command we have seen in the lecture.

In [ ]:
text = "Mr. Bob Dobolina is thinkin' of a master plan." + \
       "\nWhy doesn't he quit?"
text.split(" ")

['Mr.',
 'Bob',
 'Dobolina',
 'is',
 "thinkin'",
 'of',
 'a',
 'master',
 'plan.\nWhy',
 "doesn't",
 'he',
 'quit?']

To make more fine-grained decision, we will focus on using regular expressions for tokenization in this assignment. This can be done by either:
1. Defining the character sequence patterns at which to split.
2. Specifying patters that define what constitutes a token. 

In the code below we use a simple pattern `\s` that matches **any whitespace** to define where to split.

In [2]:
import re
gap = re.compile('\s')
gap.split(text)

NameError: name 'text' is not defined

One **shortcoming** of this tokenization is its treatment of punctuation because it considers `plan.` as a token whereas ideally we would prefer `plan` and `.` to be distinct tokens. It might be easier to address this problem if we define what a token is, instead of what constitutes a gap. Below we have defined tokens as sequences of alphanumeric characters and punctuation.

In [3]:
token = re.compile('\w+|[.?:]')
token.findall(text)

NameError: name 'text' is not defined

This still isn't perfect as `Mr.` is split into two tokens, but it should be a single token. Moreover, we have actually lost an apostrophe. Both are fixed below, although we now fail to break up the contraction `doesn't`.

In [4]:
token = re.compile('Mr.|[\w\']+|[.?]')
tokens = token.findall(text)
tokens

NameError: name 'text' is not defined

In the code below, we have an input text and apply the tokenizer (described previously) on the text:

In [5]:
import re
text = """'Curiouser and curiouser!' cried Alice (she was so much surprised, that for the moment she quite
forgot how to speak good English); 'now I'm opening out like the largest telescope that ever was! Good-bye,
feet!' (for when she looked down at her feet, they seemed to be almost out of sight, they were getting so far
off). 'Oh, my poor little feet, I wonder who will put on your shoes and stockings for you now, dears? I'm sure I
shan't be able! I shall be a great deal too far off to trouble myself about you: you must manage the best
way you can; —but I must be kind to them,' thought Alice, 'or perhaps they won't walk the way I want to go!
Let me see: I'll give them a new pair of boots every Christmas...'
"""

token = re.compile('Mr.|[\w\']+|[.?]')
tokens = token.findall(text)
print(tokens[:10])
print(len(tokens))

["'Curiouser", 'and', 'curiouser', "'", 'cried', 'Alice', 'she', 'was', 'so', 'much']
147


Questions:

* a) The tokenizer clearly makes a few mistakes. Where?

* b) Write a tokenizer to correctly tokenize the text.

* c) Should one separate `'m`, `'ll`, `n't`, possessives, and other forms of contractions from the word? Implement a tokenizer that separates these, and attaches the `'` to the latter part of the contraction.

* d) Should elipsis (...) be considered as three `.`s or one `...`? Design a regular expression for both solutions.


In [6]:
# For a & b
token2 = re.compile('^\'|\'$|\s[\'-]|[\'-]\s|Mr.|[\w\'-]+|\.+|[?!)(,:;—]')
tokens2 = token2.findall(text)
print(tokens2[:10])
print(len(tokens2))

# For c 
token3 = re.compile('^\'|\'$|\s[\'-]|[\'-]\s|Mr.|I|[sS]han|[Ww]o|[\w\'-]+|\.+|[?!)(,:;—]')
tokens3 = token3.findall(text)
print(tokens3[-20:-10])
print(len(tokens3))

# For d
token_together = re.compile('^\'|\'$|\s[\'-]|[\'-]\s|Mr.|[\w\'-]+|\.+|[?!)(,:;—]')
token_apart = re.compile('^\'|\'$|\s[\'-]|[\'-]\s|Mr.|[\w\'-]+|\.|[?!)(,:;—]')

["'", 'Curiouser', 'and', 'curiouser', '!', "' ", 'cried', 'Alice', '(', 'she']
171
['to', 'go', '!', 'Let', 'me', 'see', ':', 'I', "'ll", 'give']
177


## 3. Twitter Tokenization
As you might imagine, tokenizing tweets differs from standard tokenization. There are 'rules' on what specific elements of a tweet might be (mentions, hashtags, links), and how they are tokenized. The goal of this exercise is not to create a bullet-proof Twitter tokenizer but to understand tokenization in a different domain.

In the next exercises, we will focus on the following tweet:

In [7]:
tweet = "@robv New vids coming tomorrow #excited_as_a_child, can't w8!!"

In [8]:
token = re.compile('[\w]+')
tokens = token.findall(tweet)
print(tokens)

['robv', 'New', 'vids', 'coming', 'tomorrow', 'excited_as_a_child', 'can', 't', 'w8']


Questions:
- a) What is the correct tokenization of the tweet above according to you?
- b) Try your tokenizer from the previous exercise (Question 4). Which cases are going wrong? Make sure your tokenizer handles the above tweet correctly.
- c) Will your tokenizer correctly tokenize emojis?
- d) Think of at least one other example where your tokenizer will behave incorrectly.

In [9]:
# a) @robv New vids coming tomorrow #excited_as_a_child , can't w8 !!
# b)
token = re.compile('^\'|\'$|\s[\'-]|[\'-]\s|Mr.|[#@]*[\w\'-]+|\.+|[?!)(,:;—]+')
tokens = token.findall(tweet)
print(tokens)

['@robv', 'New', 'vids', 'coming', 'tomorrow', '#excited_as_a_child', ',', "can't", 'w8', '!!']


## 4. Segmentation


Sentence segmentation is not a trivial task either.

First, make sure you understand the following sentence segmentation code used in the lecture:

In [10]:
import re

def sentence_segment(match_regex, tokens):
    """
    Splits a sequence of tokens into sentences, splitting wherever the given matching regular expression
    matches.

    Parameters
    ----------
    match_regex the regular expression that defines at which token to split.
    tokens the input sequence of string tokens.

    Returns
    -------
    a list of token lists, where each inner list represents a sentence.

    >>> tokens = ['the','man','eats','.','She', 'sleeps', '.']
    >>> sentence_segment(re.compile('\.'), tokens)
    [['the', 'man', 'eats', '.'], ['She', 'sleeps', '.']]
    """
    current = []
    sentences = [current]
    for tok in tokens:
        current.append(tok)
        if match_regex.match(tok):
            current = []
            sentences.append(current)
    if not sentences[-1]:
        sentences.pop(-1)
    return sentences


In the following code, there is a variable `text` containing a small text and a regular expression-based segmenter:

In [11]:
text = """
Llanfairpwllgwyngyllgogerychwyrndrobwllllantysiliogogogoch is the longest official one-word placename in U.K. Isn't that weird? I mean, someone took the effort to really make this name as complicated as possible, huh?! Of course, U.S.A. also has its own record in the longest name, albeit a bit shorter... This record belongs to the place called Chargoggagoggmanchauggagoggchaubunagungamaugg. There's so many wonderful little details one can find out while browsing http://www.wikipedia.org during their Ph.D. or an M.Sc.
"""
# B
token = re.compile('[A-Z][\w\.]+|http://www\.[\w\d]+\.\w+|[\w\'-]+|[.?!,]+')

tokens = token.findall(text)
sentences = sentence_segment(re.compile('[\.?]'), tokens)
for sentence in sentences:
    print(sentence)

# C, websites; http://www\.[\w\d]+\.\w+
# C, abbreviations; [A-Z][\w\.]+

['Llanfairpwllgwyngyllgogerychwyrndrobwllllantysiliogogogoch', 'is', 'the', 'longest', 'official', 'one-word', 'placename', 'in', 'U.K.', 'Isn', "'t", 'that', 'weird', '?']
['I', 'mean', ',', 'someone', 'took', 'the', 'effort', 'to', 'really', 'make', 'this', 'name', 'as', 'complicated', 'as', 'possible', ',', 'huh', '?!']
['Of', 'course', ',', 'U.S.A.', 'also', 'has', 'its', 'own', 'record', 'in', 'the', 'longest', 'name', ',', 'albeit', 'a', 'bit', 'shorter', '...']
['This', 'record', 'belongs', 'to', 'the', 'place', 'called', 'Chargoggagoggmanchauggagoggchaubunagungamaugg.', 'There', "'s", 'so', 'many', 'wonderful', 'little', 'details', 'one', 'can', 'find', 'out', 'while', 'browsing', 'http://www.wikipedia.org', 'during', 'their', 'Ph.D.', 'or', 'an', 'M.Sc.']


Questions:
- a) Improve the segmenter so that it segments the text in the way you think it is correct.
- b) How would you deal with all URLs effectively?
- c) Can you think of other problematic cases not covered in the example?

# Competition

We host a competition on tokenizing English social media data. There is example data available for development, and instructions how to participate on: https://www.kaggle.com/competitions/twitter-tokenization

In [12]:
import pandas as pd
import csv

In [13]:
x_train = pd.read_csv('twitter-tokenization/X_train.csv')
y_train = pd.read_csv('twitter-tokenization/y_train.csv')
train = x_train.merge(y_train)
x_test = pd.read_csv('twitter-tokenization/X_test.csv')

In [138]:
abbreviation_dict = {
                    # Slang
                    "gonna":"gon na", "wanna":"wan na", "aint":"ai nt", "gotta":"got ta", "tryna":"try na", 
                        "Gonna":"Gon na", "Wanna":"Wan na", "Aint":"Ai nt", "Gotta":"Got ta", "Tryna":"Try na",
                        "GONNA":"GON NA", "WANNA":"WAN NA", "AINT":"AI NT", "GOTTA":"GOT TA", "TRYNA":"TRY NA",
                    "gimme":"gim me", "boutta":"bout ta", "kinda":"kind a", #"let's":"let 's",
                        "Gimme":"Gim me", "Boutta":"Bout ta", "Kinda":"Kind a", #"Let's":"Let 's",
                        "GIMME":"GIM ME", "BOUTTA":"BOUT TA", "KINDA":"KIND A", #"LET'S":"LET 'S",

                    # These are very much specific to the dataset (slang-language)
                    "thers":"ther s", "Thers":"Ther s", "dats":"dat s", "Dats":"Dat s",
                        "wats":"wat s", "Wats":"Wat s", "WATS":"WAT S", "dnt":"d nt",
                        #" ur ":" u r ", " Ur ":" U r ", #These are ambiguously used for "your" and "you're"
                        "yll":"y ll", "Yll":"Y ll", "YLL":"Y LL",
                        "y'all":"y 'all", "Y'all":"Y 'all", "Y'ALL":"Y 'ALL",
                            "ya'll":"ya 'll", "Ya'll":"Ya 'll", "YA'LL":"YA 'LL",
                            "yall":"y all", "Yall":"Y all", "YALL":"Y ALL",
                        "hafta":"haf ta", "Hafta":"Haf ta", "HAFTA":"HAF TA",
                    
                    # I
                    "I'm":"I 'm", "I'M":"I 'M", "i'm":"i 'm",
                        "Im":"I m", "IM":"I M", "im":"i m",
                    "I'd":"I 'd", "I'D":"I 'D", "i'd":"i 'd",
                        "Id":"I d", "ID":"I D", "id":"i d",
                    "I've":"I 've", "I'VE":"I 'VE", "i've":"i 've",
                        "Ive":"I ve", "IVE":"I VE", "ive":"i ve",
                        "I'v":"I 'v", "I'V":"I 'V", "i'v":"i 'v",
                            "Iv":"I v", "IV":"I V", "iv":"i v",
                    "I'll":"I 'll", "I'LL":"I 'LL", "i'll":"i 'll",
                        "Ill":"I ll", "ILL":"I LL", "ill":"i ll",
                    
                    # You
                    "you're":"you 're", "YOU'RE":"YOU 'RE", "You're":"You 're",
                        "youre":"you re", "YOURE":"YOU RE", "Youre":"You re",
                    "you'd":"you 'd", "YOU'D":"YOU 'D", "You'd":"You 'd",
                        "youd":"you d", "YOUD":"YOU D", "Youd":"You d",
                    "you'll":"you 'll", "YOU'LL":"YOU 'LL", "You'll":"You 'll",
                        "youll":"you ll", "YOULL":"YOU LL", "Youll":"You ll",

                    # We
                    "we're":"we 're", "WE'RE":"WE 'RE", "We're":"We 're",
                    "we'd":"we 'd", "WE'D":"WE 'D", "We'd":"We 'd",
                    "we'll":"we 'll", "WE'LL":"WE 'LL", "We'll":"We 'll",

                    # He / She
                    "he's":"he 's", "He's":"He 's", "HE'S":"HE 'S",
                        "hes":"he s", "Hes":"He s", "HES":"HE S",
                    "she's":"she 's", "She's":"She 's", "SHE'S":"SHE 'S",
                        "shes":"she s", "Shes":"She s", "SHES":"SHE S",
                    "who's":"who 's", "Who's":"Who 's", "WHO'S":"WHO 'S",
                        "whos":"who s", "Whos":"Who s", "WHOS":"WHO S",
                    "how's":"how 's", "How's":"How 's", "HOW'S":"HOW 'S",
                        "hows":"how s", "Hows":"How s", "HOWS":"HOW S",
                    "that's":"that 's", "That's":"That 's", "THAT'S":"THAT 'S",
                        "thats":"that s", "Thats":"That s", "THATS":"THAT S",
                    "what's":"what 's", "What's":"What 's", "WHAT'S":"WHAT 'S",
                        "whats":"what s", "Whats":"What s", "WHATS":"WHAT S",
                    "where's":"where 's", "Where's":"Where 's", "WHERE'S":"WHERE 'S",
                        "wheres":"where s", "Wheres":"Where s", "WHERES":"WHERE S",
                    "there's":"there 's", "There's":"There 's", "THERE'S":"THERE 'S",
                        "theres":"there s", "Theres":"There s", "THERES":"THERE S",
                    "its":"it s", "Its":"It s",

                    # It / They
                    "its":"it s", "Its":"It s", "it's":"it 's", "It's":"It 's", 
                        "ITS":"IT S","IT'S":"IT 'S","i'z":"i 'z",
                    "itll":"it ll", "Itll":"It ll", "it'll":"it 'll", "It'll":"It 'll", 
                        "ITLL":"IT LL","IT'LL":"IT 'LL",
                    "They'll":"They 'll", "THEY'LL":"THEY 'LL", "they'll":"they 'll",
                        "Theyll":"They ll", "THEYLL":"THEY LL", "theyll":"they ll",
                    "They'd":"They 'd", "THEY'D":"THEY 'D", "they'd":"they 'd",
                        "Theyd":"They d", "THEYD":"THEY D", "theyd":"they d",
                    "They're":"They 're", "THEY'RE":"THEY 'RE", "they're":"they 're",
                        "Theyre":"They re", "THEYRE":"THEY RE", "theyre":"they re",

                    # Negations
                    "can't":"ca n't", "CAN'T":"CA N'T", "Can't":"Ca n't", 
                        "cant":"ca nt", "Cant":"Ca nt", "CANT":"CA NT",
                    "won't":"wo n't", "WON'T":"WO N'T", "Won't":"Wo n't",
                        "wont":"wo nt", "Wont":"Wo nt", "WONT":"WO NT",
                    "don't":"do n't", "DON'T":"DO N'T", "Don't":"Do n't",
                        "dont":"do nt", "Dont":"Do nt", "DONT":"DO NT",
                    "didn't":"did n't", "DIDN'T":"DID N'T", "Didn't":"Did n't",
                        "didnt":"did nt", "Didnt":"Did nt", "DIDNT":"DID NT",
                    "wasn't":"was n't", "WASN'T":"WAS N'T", "Wasn't":"Was n't",
                        "wasnt":"was nt", "Wasnt":"Was nt", "WASNT":"WAS NT",
                    "doesn't":"does n't", "DOESN'T":"DOES N'T", "Doesn't":"Does n't",
                        "doesnt":"does nt", "Doesnt":"Does nt", "DOESNT":"DOES NT",
                    
                    # Writing out the meaning of the abbreviation / slang:
                    "Ima":"Ima I am going to", "ima":"ima I am going to", 
                    "Imma":"Imma I am going to", "imma":"imma I am going to"
                    }

difficult_abbreviations = {
    # For the extra check when first word is im / its 
    "its":"it s", "Its":"It s",
    "Im":"I m", "IM":"I M", "im":"i m",
    "Ive":"I ve", "IVE":"I VE", "ive":"i ve",
    "Id":"I d", "ID":"I D", "id":"i d",
    "Iv":"I v", "IV":"I V", "iv":"i v",
    "Ill":"I ll", "ILL":"I LL", "ill":"i ll",
}

In [139]:
# Additional improvements; 
### find more character emojis; ;o :S
### find more regular abbreviations
pattern = str('[Dd)(pP/]*-?[:;=]-?[Dd)(pP/]*|[\u263a-\U0001f645]|<3'+ #Emojis
    '|http://[\./\w\d]+|www\.[\w\d]+\.\w+/?\w*|[\w\d]+\.com[/\w]*'+ #Websites
    '|\d[a-z]+'+ #Words starting with a number
    '|\d+[/:-]\d+/?\d*|[$]?\d+[.,]?\d*[Tt]?[Hh]?'+ #Formatted Numbers
    '|\'em|\'|w/' + # Starting Quote
    '|[#@]*[\w`\'-]+'+ #Regular words + hashtags / user tags
    '|\.+|[?!,;:—"&/*<>\.]+|[«»)(@“”+%]'+ #Punctuation etc. 
    '|[^\s]') # This is a Catch all!
token = re.compile(pattern)

my_tokens = []
for i in range(train.shape[0]):
    data = train.iloc[i]['Text']
    tokens = token.findall(data)
    for j, tok in enumerate(tokens):
        if tok in abbreviation_dict:
            tokens[j] = abbreviation_dict[tok]
    if tokens[0] in difficult_abbreviations:
        tokens[0] = difficult_abbreviations[tokens[0]]
    my_tokens.append(" ".join(tokens))

train['My Tokens'] = my_tokens

['www.my_web.com/123']


In [140]:
wrong = train[train['My Tokens'] != train['Expected']]
wrong[['Id', 'Expected', 'My Tokens']].to_csv('tokenized_wrong.csv')

In [141]:
wrong.shape[0]

33

In [142]:
preds = []
for i in range(x_test.shape[0]):
    data = x_test.iloc[i]['Text']
    tokens = token.findall(data)
    for j, tok in enumerate(tokens):
        if tok in abbreviation_dict:
            tokens[j] = abbreviation_dict[tok]
    if tokens[0] in difficult_abbreviations:
        tokens[0] = difficult_abbreviations[tokens[0]]
    preds.append(" ".join(tokens))

x_test['Predicted'] = preds
x_test[['Id', 'Predicted']].to_csv('test_preds.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

# Lecture 2

## 5. Linux Command Line for NLP: Conll Format
In natural language processing, the "conll" format is a highly common standard to represent annotated text. There is a variety of conll formats, so it might be more correct to refer to them as conll-like formats. These formats have one word per line, separate sentences with an empty line, and have separate collumns (separated with tabs) for each annotation layer.

In this assignment, we will use the conll format for named entity recognition (from conll2002: [paper](https://aclanthology.org/W02-2024.pdf)). We will use Danish data from (DaN+)[https://aclanthology.org/2020.coling-main.583.pdf]. This data follows the BIO labels as discussed in the lecture. An example of the data is shown below, this example has one entity-span "goergh bush":

```
-       O
en      O
mand    O
der     O
hedder  O
goergh  B-PER
bush    I-PER
.       O

```


Use Unix command line tools for this assignment (grep, sed, etc.)

* a) Search in the `da_arto.conll` file (in the assingment1 directory) for first names. You can assume that first names always have the label B-PER, and that the string "B-PER" does not occur in the first column. 
* b) How many names occur in the data?
* c) How can we make sure that we do not match the string "B-PER" occuring in the first column?
* d) How can we clean away the labels, so that we have only a list of names left? (hint: pipe the result of the previous command into a `split`)
* e) How many of the names you found start with an uppercased character?




### A
grep 'B-PER' da_arto.conll | cut -f 1

### B (96!)
grep 'B-PER' da_arto.conll | cut -f 1 | wc -w

### C --- 
Get back to me :)
Maybe i was supposed to do the cut only at this point in time?

### D
grep "B-PER" da_arto.conll | cut -f 1 | grep "^[A-Z]"

### E (53!)
grep "B-PER" da_arto.conll | cut -f 1 | grep "^[A-Z]" | wc -w

## 6. More Advanced Usage of Unix Tools: Creating a word frequency list, finding function words
Let us now create a simple word frequency list from the book above using Unix tools to answer the following question: Which four (function) words are the most frequent in *The Adventures of Sherlock Holmes* by Arthur Conan Doyle (`pg1661.txt`)?

* The first step is to split the text into separate words. Here, we will use the command sed to replace all spaces with a newline:

```
sed 's/ /\n/g' FILE
```

Note: Remember the flag `g`, which stands for global. It replaces all occurrences of a space on a line.

* Hint: It is handy to forward this command to a tool called `less`, which lets you browse through the result (type `q` to quit).

```
sed 's/ /\n/g' FILE | less
```

* Now we can sort the list of tokens and count unique words:

```
sed 's/ /\n/g' FILE | sort | uniq -c
```

* To create the most frequent words first, sort again in reverse numeric order (find the options of `sort` to do so, e.g. check `man sort`).

Note: Here we used `sed`, our textbook shows an alternative with `tr` instead.